In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# packages
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time

# from goatools import obo_parser

# 1. Pubtator API (TODO)
- using the input folder, check if the file(s) have been pre-annotated by checking if the following APIs return something
```
https://www.ncbi.nlm.nih.gov/research/pubtator3-api/publications/export/biocxml?pmids={id}
or
https://www.ncbi.nlm.nih.gov/research/pubtator3-api/publications/pmc_export/biocxml?pmcids=PMC{id}
```
  - if annotated, retrieve it and put it in the output folder
  - if not, use the example code they provide to send it to the server to be annotated

# 2. Relevance (TODO)
- This can be integrated with Gene Chunking. We can just check if the paper has genes (i.e check if there is an item in the dictionary)

*3. Gene Chunking*
=============
Given a **preannotated** article (check input folder) analyze the **XML** file via `parse_xml_file(xml_file_path)` which returns a dictionary of "Gene Class" from that file  

The Gene Class has the following information:
- self.gene_id: the gene id (from the xml file)
- self.occurences: A list of occurences of the gene in the text. Each occurence is **3-sentences long**.
    - The second sentence contains the explicit mention of the gene.
    - The first and third sentences are the context of the gene mention.
- self.symbol: the gene symbol (from NCBI. Initialized to None)
- self.organism: the organism of the gene (from NCBI. Initialized to None)
- self.full_name: the full name of the gene (from NCBI. Initialized to None. Set to name_from_article if not found in NCBI)
- self.also_known_as: a list of other names of the gene (from NCBI. Initialized to None. Set to name_from_article if not found in NCBI)
- self.name_from_article: the name of the gene as found in the article

Steps:
1. define the class Gene
2. Parse the xml file
3. update the Gene class with the information from NCBI
Now you have a dictionary of Gene classes.~

TIPS:
- control + F "sentence_buffer" to find where you can adjust the buffer

In [68]:
!pip install biopython
import xml.etree.ElementTree as ET
from Bio import Entrez
import re
import time

class Gene:
    def __init__(self, gene_id):
        self.gene_id = gene_id
        self.occurrences = []  # list to store snippet(s) where the gene was mentioned
        self.symbol = None
        self.organism = None
        self.full_name = None
        self.also_known_as = None
        self.name_from_article = None
        # TODO: add pmid/pmcid which will be used for final output. 

    def add_occurrence(self, snippet):
        if snippet not in self.occurrences:  # Avoid duplicates
            self.occurrences.append(snippet)

    def set_name_from_article(self, name_from_article):
        """Sets the temporary name of the gene. This is the name accroding to the article"""
        # will be used if the official name is not available
        self.name_from_article = name_from_article

    def get_name_from_article(self):
        return self.name_from_article

    def get_occurrences(self):
        return self.occurrences

    def get_also_known_as(self):
        return self.also_known_as

    def get_gene_id(self):
        return self.gene_id

    def update_info(self, symbol, organism, full_name, also_known_as):
        self.symbol = symbol
        self.organism = organism
        self.full_name = full_name
        self.also_known_as = also_known_as

    def __repr__(self):
        return (f"Gene({self.gene_id})\n"
                f"  Symbol          : {self.symbol}\n"
                f"  Organism        : {self.organism}\n"
                f"  Full Name       : {self.full_name}\n"
                f"  Also Known As   : {self.also_known_as}\n"
                f"  In-text Name    : {self.name_from_article}\n"
                f"  Occurrences     : {self.occurrences}")   # occurrences is a list of 3-sentence snippets

def parse_xml_file(xml_path):
    """Parses the XML file and returns a gene dictionary keyed by gene ID."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    gene_dict = {}

    for document in root.findall('document'):
        for passage in document.findall('passage'):
            section_type_elem = passage.find("infon[@key='section_type']")
            if section_type_elem is not None and (section_type_elem.text.upper() == "METHODS") or (section_type_elem.text.upper() == "FIG") or (section_type_elem.text.upper() == "TABLE"):
                continue  # Skip passages under METHODS
            # Get the full passage text.
            passage_text_elem = passage.find("text")
            passage_text = passage_text_elem.text if passage_text_elem is not None else ""
            # determine the starting offset for this passage.
            passage_offset_elem = passage.find("offset")
            passage_offset = int(passage_offset_elem.text) if passage_offset_elem is not None else 0
            # Split the passage into sentences using regex.
            sentences = re.split(r'(?<=[.!?])\s+', passage_text)
            # Compute start indices for each sentence within the passage text.
            start_indices = []
            current_index = passage_offset
            for sentence in sentences:
                start_indices.append(current_index)
                current_index += len(sentence) + 1  # account for the delimiter space

            processed_ranges = set()  # To track which sentences have already been covered

            # Process each annotation in the passage.
            for annotation in passage.findall('annotation'):
                ann_type = annotation.find("infon[@key='type']")
                if ann_type is not None and ann_type.text == "Gene":
                    # Check for both 'identifier' and 'NCBI Gene' keys
                    gene_id_elem = annotation.find("infon[@key='identifier']")
                    if gene_id_elem is None:
                        gene_id_elem = annotation.find("infon[@key='NCBI Gene']")
                    gene_id = gene_id_elem.text if gene_id_elem is not None else None

                    # Extract the gene name from the annotation text. (Temporary name if official name is not available)
                    in_text_gene_name_elem = annotation.find("text")
                    in_text_gene_name = in_text_gene_name_elem.text if in_text_gene_name_elem is not None else None

                    # Extract the annotation location (offset) to find the sentence containing the gene.
                    location_elem = annotation.find("location")
                    ann_offset = int(location_elem.attrib.get('offset', 0)) if location_elem is not None else 0

                    # Determine which sentence contains the annotation based on its offset.
                    sentence_index = None
                    for i, start in enumerate(start_indices):
                        if start <= ann_offset < start + len(sentences[i]):
                            sentence_index = i
                            break

                    # sentence_buffer
                    if sentence_index is not None:
                        start_sentence = max(0, sentence_index - 1)  # one sentence before
                        end_sentence = min(len(sentences), sentence_index + 2)  # one sentence after
                        range_tuple = (start_sentence, end_sentence)
                        if range_tuple in processed_ranges:
                            continue  # Skip duplicate extractions
                        processed_ranges.add(range_tuple)
                        snippet = " ".join(sentences[start_sentence:end_sentence])
                        if gene_id:
                            if gene_id in gene_dict:
                                gene_dict[gene_id].add_occurrence(snippet)
                            else:
                                gene_obj = Gene(gene_id)
                                gene_obj.add_occurrence(snippet)
                                gene_obj.set_name_from_article(in_text_gene_name)       # Temporary name if official name is not available
                                gene_dict[gene_id] = gene_obj
    return gene_dict

def fetch_and_update_gene_info(gene_dict):
    """Retrieves gene information from NCBI and updates the genes in gene_dict."""
    gene_ids = list(gene_dict.keys())
    if gene_ids:
        # Post the gene IDs to NCBI.
        handle = Entrez.epost(db="gene", id=",".join(gene_ids))
        result = Entrez.read(handle)
        handle.close()

        webenv = result["WebEnv"]
        query_key = result["QueryKey"]

        handle = Entrez.esummary(db="gene", webenv=webenv, query_key=query_key)
        record = Entrez.read(handle)
        handle.close()

        for docsum in record["DocumentSummarySet"]["DocumentSummary"]:
            gene_id = docsum.attributes["uid"]
            symbol = docsum.get('NomenclatureSymbol', 'No symbol')
            organism = docsum.get('Organism', {}).get('ScientificName', 'No organism')
            full_name = docsum.get('NomenclatureName', gene_dict[gene_id].get_name_from_article())
            also_known_as = docsum.get('OtherAliases', gene_dict[gene_id].get_name_from_article())

            # if full name and also known as are not available, use the name from the article
            if (full_name == ''):
                full_name = gene_dict[gene_id].get_name_from_article()
            if (also_known_as == ''):
                also_known_as = gene_dict[gene_id].get_name_from_article()

            if gene_id in gene_dict:
                gene_dict[gene_id].update_info(symbol, organism, full_name, also_known_as)
            # Pause briefly to avoid overwhelming NCBI servers.
            time.sleep(0.5)     # 2 requests per second (safe). WITH API KEY, be increased if needed to 10 requests per second.


# Set your email (and API key if available)
Entrez.email = "email here"
# Entrez.api_key = "your_api_key"

# example2.xml takes a while since it has 64 genes.
xml_path = "/content/full_text_annotated_example.xml"
gene_dict = parse_xml_file(xml_path)
fetch_and_update_gene_info(gene_dict)           # Fetch gene information from NCBI (Optional but has good information)
for gene in gene_dict.values():
    print(gene)
    print(f"There are {len(gene.get_occurrences())} occurrences (each are AT MOST 3-setences long SO MAX of {int(3*len(gene.get_occurrences()))} sentences in total) of {gene.get_gene_id()}:{gene.get_name_from_article()}.")
    print("")

Gene(2670)
  Symbol          : GFAP
  Organism        : Homo sapiens
  Full Name       : glial fibrillary acidic protein
  Also Known As   : ALXDRD
  In-text Name    : glial fibrillary acidic protein
  Occurrences     : ['We found that a lectin, Datura stramonium agglutinin, induced irreversible differentiation in C6 glioma cells. The differentiated cells had long processes, a low rate of proliferation and a high content of glial fibrillary acidic protein. When the medium was replaced with Datura stramonium agglutinin-free medium after 1 h, cell proliferation continued to be inhibited.', 'Proliferation of four human glial tumour cells was also inhibited by Datura stramonium agglutinin. Further, these differentiated human glial tumour cells had long processes and a high content of glial fibrillary acidic protein similar to differentiated C6 glioma cells. Taken together, these observations suggest that Datura stramonium agglutinin may be useful as a new therapy for treating glioma withou

# 3.5 Prompt Engineering
- Creates the prompt using the Gene dictionary (from Gene chunking)
- batch_size: allows you to have more than 1 Gene in the prompt such that you have something like the following. Ideally keep it at batch_size=1 to ensure that the GO terms generated are exclusive to the Gene being analyzed.

```
# batch_size > 1 has the basic format:
Basic Prompt
Gene 1
Gene 2
...
# batch_size = 1 has the basic format:
Basic Prompt
Gene 1
```
- batch_index: allows you to iterate through the gene dictionary so that if you have a paper with x amount of genes, you can just increment  the index and run the inference x amount of times.
- direct: ignore this. This is for the future.
- customized_prompt: ignore this as well. This is for the future.

In [69]:
def make_go_term_prompt(gene_dict, batch_size=1, batch_index=0, direct=False, customized_prompt=None):
    """
    Create a prompt for Llama to output Gene Ontology (GO) terms (without IDs) for a batch of genes
    from the gene dictionary. The output is expected to be categorized into three sections for each gene:
      1. Biological Processes
      2. Cellular Components
      3. Molecular Functions

    For each gene, the output should follow the format below without mixing results across different genes.

    :param gene_dict: A dictionary where each key is a gene ID and each value is a Gene object that
                      contains attributes such as symbol, organism, full_name, and occurrences.
    :param batch_size: The number of genes to include in this prompt batch.
    :param batch_index: The index (0-based) of the batch to process. For instance, if batch_size=1,
                        batch_index=2 will include the 3rd gene in the dictionary.
    :param direct: If True, use a more direct instruction for GO term extraction.
    :param customized_prompt: If provided, use this custom prompt text in place of the default.
    :return: A string containing the complete prompt.
    """

    # Instruction blocks

    # context = """You are an efficient and insightful assistant to a geneticist."""

    general_instructions = """Analyze the following gene information and text snippets under "Occurrences".
Identify all potentially relevant Gene Ontology (GO) terms.
Base your analysis on prior knowledge available in your training data.

After completing your analysis, for each Gene Ontology (GO) term you list, provide a short one-line justification referencing the text as well as a relationship between the gene and the term.
If you cannot identify any GO terms for a particular gene information, say “None found.”

Be concise, do not use unnecessary words.
Be factual, do not editorialize.
Be specific, avoid overly general statements like "it is involved in many cellular processes.

"""

    example = """To help you in your work, I am providing an example of gene information and the corresponding example analysis output structured in JSON.

Example gene information is:
Gene: myogenin
Organism: Mus musculus
Occurrences:
- Activation of NF-kappaB increases the expression of the inducible nitric oxide synthase (iNOS) in muscle cells that sequester HuR (RNA-binding protein) by preventing transcription of MyoD. Denervation-induced atrophy is related to the upregulation of specific histone deacetylases (HDAC), able to repress a negative regulator of myogenin, with an increase in MuRF1 expression and muscle wasting. AMPK (protein kinase activated by 5'adenosine monophosphate) is also involved in muscle atrophy.

Example anaylysis output is:
[
  {
    "gene": "myogenin",
    "go_data": [
      {
        "term": "DNA-binding transcription activator activity",
        "relationship": "enables",
        "namespace": "molecular_function",
        "justification": "Myogenin is a key transcription factor in muscle differentiation that binds to E-box sequences in muscle-specific genes and activates their transcription through RNA polymerase II."

      },
      {
        "term": "skeletal muscle cell differentiation",
        "relationship": "involved_in",
        "namespace": "biological_process"
        "justification": "Myogenin is one of the four myogenic regulatory factors (MRFs) that drive skeletal muscle differentiation by promoting the transition from myoblasts to myotubes."
      }]
    }
]
"""


    # Assemble the prompt text based on the chosen mode
    if direct:
        # prompt_text = context
        # prompt_text += direct_instructions
        # prompt_text += format_instructions
        # prompt_text += example_output
        pass
    elif customized_prompt:
        # prompt_text = context
        # prompt_text += customized_prompt
        # prompt_text += format_instructions
        # prompt_text += example_output
        pass
    else:
        prompt_text = general_instructions
        prompt_text += example
        prompt_text += "\nHere is the gene information:\n"

    # Convert the gene dictionary to a list and determine the batch slice.
    gene_items = list(gene_dict.items())
    start_index = batch_index * batch_size
    end_index = start_index + batch_size
    batch_genes = gene_items[start_index:end_index]

    # Add details and occurrence context for each gene in the selected batch.
    for gene_id, gene_obj in batch_genes:
        # prompt_text += f"\nGene ID: {gene_id}\n"
        # prompt_text += f"Symbol: {gene_obj.symbol}\n"
        # prompt_text += f"Organism: {gene_obj.organism}\n"
        # prompt_text += f"Full Name: {gene_obj.full_name}\n"
        prompt_text += f"\nGene: {gene_obj.get_name_from_article()}\n"
        prompt_text += "Occurrences:\n"
        for occ in gene_obj.occurrences:
            prompt_text += f"- {occ}\n"

    return prompt_text

# Example usage:
# Assuming gene_dict is defined and contains Gene objects with attributes: symbol, organism, full_name, and occurrences.
# To process only one gene at a time:
# prompt = make_go_term_prompt(gene_dict, batch_size=1, batch_index=0)
# print(prompt)

prompt = make_go_term_prompt(gene_dict)
print(prompt)

Analyze the following gene information and text snippets under "Occurrences".
Identify all potentially relevant Gene Ontology (GO) terms.
Base your analysis on prior knowledge available in your training data.

After completing your analysis, for each Gene Ontology (GO) term you list, provide a short one-line justification referencing the text as well as a relationship between the gene and the term.
If you cannot identify any GO terms for a particular gene information, say “None found.”

Be concise, do not use unnecessary words.
Be factual, do not editorialize.
Be specific, avoid overly general statements like "it is involved in many cellular processes.

To help you in your work, I am providing an example of gene information and the corresponding example analysis output structured in JSON.

Example gene information is:
Gene: myogenin
Organism: Mus musculus
Occurrences:
- Activation of NF-kappaB increases the expression of the inducible nitric oxide synthase (iNOS) in muscle cells that s

# 4. Inference (Local)
The two cells below rely on local inference meaning that you must have the model locally. Skip them if youre using the API like GROQ

In [ ]:
print("Loading LLaMA model.")
# model_dir = "/content/drive/MyDrive/Llama 3.2-3B-Instruct-model"
model_dir = "/content/drive/MyDrive/GOLLM/Llama 3.2-3B-Instruct-model"
device = torch.device("cuda")
print(torch.cuda.is_available())

tokenizer = AutoTokenizer.from_pretrained(model_dir, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_dir, local_files_only=True).to(device)


Loading LLaMA model.
False


OSError: Incorrect path_or_model_id: '/content/drive/MyDrive/GOLLM/Llama 3.2-3B-Instruct-model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
def run_inference(prompt):
    """
    Run inference on the provided prompt using the specified model and tokenizer.

    :param prompt: The prompt string to send to the model.
    :return: The generated text.
    """

    # Encode the prompt and move to the appropriate device
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    attention_mask = inputs["attention_mask"]

    # Greedy or minimal variation == When: You want consistent, short answers, and you don’t mind if it’s a bit “blunt.”
    # outputs = model.generate(
    #     input_ids=inputs["input_ids"],
    #     attention_mask=attention_mask,
    #     max_new_tokens=150,
    #     do_sample=False,     # no sampling
    #     num_beams=1,         # purely greedy
    #     no_repeat_ngram_size=3,
    #     pad_token_id=tokenizer.eos_token_id
    # )

    # Beam Search (Higher Quality / Less Randomness) == You want a more “global optimum” text.
    # outputs = model.generate(
    #     input_ids=inputs["input_ids"],
    #     attention_mask=attention_mask,
    #     max_new_tokens=200,
    #     do_sample=False,
    #     num_beams=4,      # search multiple beams
    #     length_penalty=1.0,  # see if you want to encourage or discourage long outputs
    #     no_repeat_ngram_size=3,
    #     pad_token_id=tokenizer.eos_token_id
    # )

    # Beam SearchV2
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=attention_mask,
        max_new_tokens=400,       # Enough tokens for a concise but thorough answer
        do_sample=False,          # Turn off sampling; we want a more deterministic, stable answer
        num_beams=3,              # Explore multiple beams for higher-quality completions
        length_penalty=0.1,       # 1.0 means "neutral" length preference (>=1.0 encourages longer outputs)
        no_repeat_ngram_size=4,   # Helps avoid repeating the same phrase
        early_stopping=True,      # Stops as soon as the best beam is complete
        pad_token_id=tokenizer.eos_token_id,
        temperature=None,
        top_p=None,
    )
    # Sampling (More Creative / Less Deterministic)
    # outputs = model.generate(
    #     input_ids=inputs["input_ids"],
    #     attention_mask=attention_mask,
    #     max_new_tokens=200,
    #     do_sample=True,       # sampling
    #     temperature=0.7,      # moderate creativity
    #     top_p=0.9,            # nucleus sampling
    #     no_repeat_ngram_size=3,
    #     pad_token_id=tokenizer.eos_token_id
    # )

    # Decode the output and return the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    completion = generated_text[len(prompt):].strip()
    return completion

for i in range(3):
    start_time = time.time()
    output = run_inference(test_prompt)
    print(output)

    # with open(f'/content/drive/MyDrive/GOLLM/Inference Outputs/output_{i+1}.txt', 'w') as file:
    #     file.write(output)

    print(f"-->Model loaded on {device} named {torch.cuda.get_device_name(0)} and it took {time.time()-start_time:.2f} seconds.")
    print("-"*50)

# prompt = "what is 2+2"
# print("Running inference...")
# start_time = time.time()
# output = run_inference(prompt)
# print(f"Model loaded on {device} in {time.time()-start_time:.2f} seconds.")
# print(output)


NameError: name 'test_prompt' is not defined

# 4v2. GROQ API Inference
- need API Key
- uses llama 3.3-70b-versatile

In [31]:
# using GROQ API
!pip install Groq
from groq import Groq

client = Groq(
    api_key="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"
)


In [70]:
# Generate the prompt for the gene (ensure gene_dict is defined)
prompt = make_go_term_prompt(gene_dict, batch_index=0)  # remember to increment the index for other genes

# Request a completion for gene analysis
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_completion_tokens=2048,
    top_p=0.5,
    stream=False,
    response_format={"type": "json_object"},
    stop=None,
)

# Save the output in a variable
llama_output = chat_completion.choices[0].message.content

# Print the output from the model
print(llama_output)

# Optionally, write the output to a JSON file for later use
with open("llama_output.json", "w") as f:
    f.write(llama_output)


{
   "gene":"glial fibrillary acidic protein",
   "go_data":[
      {
         "term":"intermediate filament",
         "relationship":"enables",
         "namespace":"cellular_component",
         "justification":"GFAP is expressed exclusively in astrocytes and is a type of intermediate filament protein."
      },
      {
         "term":"astrocyte differentiation",
         "relationship":"involved_in",
         "namespace":"biological_process",
         "justification":"GFAP is considered to be important for the induction and maintenance of astrocyte differentiation."
      },
      {
         "term":"regulation of cell proliferation",
         "relationship":"involved_in",
         "namespace":"biological_process",
         "justification":"Introduction of GFAP to C6 cells and human astrocytoma induced morphological changes, and suppressed their proliferation."
      },
      {
         "term":"cytoskeleton organization",
         "relationship":"involved_in",
         "namespace":

# 5. Normalize
- use an embedding model (SAPBERT) to ensure that the terms generated by Llama are actual GO terms. You will need the Go.obo file from the [Gene Ontology Consortium](https://geneontology.org/docs/download-ontology/)
  - Essentially, for each go term created by Llama, Sapbert will do a similarity search against the Go.obo and retrieve the actual term + GO:ID

In [8]:
# Install required packages (run these in your environment if not already installed)
!pip install sentence-transformers
!pip install obonet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [23]:
import obonet
import json
from sentence_transformers import SentenceTransformer, util
import numpy as np

##########################################
# 1. Load and parse the GO.obo file
##########################################

def load_go_ontology(obo_filepath):
    """
    Parse the GO.obo file using obonet.
    Returns a dictionary mapping GO IDs to their term names.
    """
    graph = obonet.read_obo(obo_filepath)
    go_terms = {}
    for go_id, data in graph.nodes(data=True):
        # Only include nodes that have a 'name' field
        if 'name' in data:
            go_terms[go_id] = data
    return go_terms

# set the path to your local go.obo (or something else later) file
obo_file = "/content/drive/MyDrive/Colab Notebooks/go.obo"
go_terms_dict = load_go_ontology(obo_file)
print(f"Loaded {len(go_terms_dict)} GO terms from {obo_file}")

Loaded 40267 GO terms from /content/drive/MyDrive/Colab Notebooks/go.obo


In [10]:
##########################################
# 2. Load the SAPBERT embedding model (or an alternative)
##########################################

# Here we use a SAPBERT model available from Hugging Face.
# may choose another model (e.g., "sentence-transformers/all-MiniLM-L6-v2")
model_name = "cambridgeltl/SapBERT-from-PubMedBERT-fulltext"  # SAPBERT model fine-tuned on UMLS
sapbert_model = SentenceTransformer(model_name)
print(f"Loaded embedding model: {model_name}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Loaded embedding model: cambridgeltl/SapBERT-from-PubMedBERT-fulltext


In [24]:
def embed_go_terms(go_terms, model):
    """
    Given a dictionary of GO terms (GO ID -> term data) and an embedding model,
    returns:
      - a list of GO IDs,
      - a list of GO term names (in the same order),
      - a tensor of embeddings for the GO term names,
      - a list of GO namespaces,
      - a list of GO relationships (if available, otherwise default to 'unknown')
    """
    go_ids = []
    go_names = []
    go_namespaces = []
    go_relationships = []

    for go_id, data in go_terms.items():
        if 'name' in data:
            go_ids.append(go_id)
            go_names.append(data['name'])
            go_namespaces.append(data.get('namespace', 'unknown'))
            # Some GO entries might not have a "relationship" field; default to 'unknown'
            go_relationships.append(data.get('relationship', 'unknown'))

    go_embeddings = model.encode(go_names, convert_to_tensor=True)
    return go_ids, go_names, go_embeddings, go_namespaces, go_relationships

# Reload GO ontology using the updated load_go_ontology:
go_terms_dict = load_go_ontology(obo_file)
go_ids, go_names, go_embeddings, go_namespaces, go_relationships = embed_go_terms(go_terms_dict, sapbert_model)
print("Embedded GO terms with additional metadata.")


Embedded GO terms with additional metadata.


In [71]:
import torch

def get_best_matching_go_term(llama_term, go_ids, go_names, go_embeddings, model, threshold=0.6):
    """
    Given a GO term from Llama, compute its embedding and find the best matching GO term
    from the provided GO ontology embeddings.

    Parameters:
      - llama_term: The GO term string generated by Llama.
      - go_ids, go_names, go_embeddings: Outputs from embed_go_terms().
      - model: The embedding model.
      - threshold: Minimum cosine similarity required to accept the match.

    Returns:
      - best_go_id: The GO:ID of the best match (or None if score is below threshold)
      - best_go_name: The GO term name of the best match.
      - best_score: The cosine similarity score.
    """
    # Encode the query using the model
    query_embedding = model.encode(llama_term, convert_to_tensor=True)
    # Compute cosine similarities between the query and all GO term embeddings
    cos_scores = torch.squeeze(util.cos_sim(query_embedding, go_embeddings))
    # Use torch.argmax to find the best matching index
    best_idx = torch.argmax(cos_scores)
    best_score = cos_scores[best_idx].item()

    if best_score >= threshold:
        return go_ids[best_idx], go_names[best_idx], best_score, best_idx
    else:
        return None, None, best_score, None

# Test the matching function with an example Llama-generated term:
example_llama_term = "muscle cell differentiation"
best_go_id, best_go_name, score, best_idx = get_best_matching_go_term(example_llama_term, go_ids, go_names, go_embeddings, sapbert_model)
print(f"Example: Llama term '{example_llama_term}' matched with '{best_go_name}' ({best_go_id}) with score {score:.2f}")


Example: Llama term 'muscle cell differentiation' matched with 'muscle cell differentiation' (GO:0042692) with score 1.00


In [72]:
import json

def parse_llama_output(llama_output_path):
    """
    Parses Llama's output JSON file.
    Returns the parsed JSON data (a list of gene entries).
    """
    try:
        with open(llama_output_path, 'r') as f:
            data = json.load(f)
        # If the JSON is a single dictionary, convert it to a list
        if isinstance(data, dict):
            data = [data]
        return data
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        return []

print(parse_llama_output("/content/llama_output.json"))

[{'gene': 'glial fibrillary acidic protein', 'go_data': [{'term': 'intermediate filament', 'relationship': 'enables', 'namespace': 'cellular_component', 'justification': 'GFAP is expressed exclusively in astrocytes and is a type of intermediate filament protein.'}, {'term': 'astrocyte differentiation', 'relationship': 'involved_in', 'namespace': 'biological_process', 'justification': 'GFAP is considered to be important for the induction and maintenance of astrocyte differentiation.'}, {'term': 'regulation of cell proliferation', 'relationship': 'involved_in', 'namespace': 'biological_process', 'justification': 'Introduction of GFAP to C6 cells and human astrocytoma induced morphological changes, and suppressed their proliferation.'}, {'term': 'cytoskeleton organization', 'relationship': 'involved_in', 'namespace': 'biological_process', 'justification': 'GFAP is a type of intermediate filament protein that provides structural support to astrocytes.'}, {'term': 'central nervous system de

In [73]:
def auto_match_go_terms(llama_data, go_ids, go_names, go_embeddings, go_namespaces, go_relationships, model, threshold=0.6):
    """
    Given parsed Llama JSON data (with keys "gene" and "go_data" containing GO term details),
    match each Llama-generated GO term (using its "term" field) to the best official GO term from GO.obo.

    For each gene, returns a list of unique matches. Each match includes:
      - The original Llama term.
      - The matched official GO term (ID and name).
      - The similarity score.
      - The official relationship and namespace from GO.obo.
      - The justification from the Llama output.

    Only matches with a similarity above the threshold are kept.
    """
    matched_go_terms = {}

    for gene_entry in llama_data:
        gene_name = gene_entry.get("gene", "UnknownGene")
        go_data = gene_entry.get("go_data", [])
        unique_matches = {}

        for entry in go_data:
            llama_term = entry.get("term")
            if not llama_term:
                continue

            # Use the Llama term to find the best matching GO term.
            best_go_id, best_go_name, best_score, best_idx = get_best_matching_go_term(
                llama_term, go_ids, go_names, go_embeddings, model, threshold=threshold
            )

            # Only add the match if it meets the threshold.
            if best_go_id and best_idx is not None:
                official_relationship = go_relationships[best_idx]
                official_namespace = go_namespaces[best_idx]

                # If this GO term was already matched, keep the one with the higher score.
                if best_go_id in unique_matches:
                    if best_score > unique_matches[best_go_id]["similarity"]:
                        unique_matches[best_go_id] = {
                            "llama_term": llama_term,
                            "matched_go_id": best_go_id,
                            "matched_go_name": best_go_name,
                            "similarity": best_score,
                            "relationship": official_relationship,
                            "namespace": official_namespace,
                            "justification": entry.get("justification")
                        }
                else:
                    unique_matches[best_go_id] = {
                        "llama_term": llama_term,
                        "matched_go_id": best_go_id,
                        "matched_go_name": best_go_name,
                        "similarity": best_score,
                        "relationship": official_relationship,
                        "namespace": official_namespace,
                        "justification": entry.get("justification")
                    }

        matched_go_terms[gene_name] = list(unique_matches.values())

    return matched_go_terms

# Usage example:
llama_output_path = "/content/llama_output.json"
llama_data = parse_llama_output(llama_output_path)

matched_terms = auto_match_go_terms(
    llama_data, go_ids, go_names, go_embeddings, go_namespaces, go_relationships, sapbert_model, threshold=0.6
)

# Output the results for each gene:
for gene, matches in matched_terms.items():
    print(f"Gene: {gene}")
    for match in matches:
        print(f"  Llama Term: {match['llama_term']}")
        print(f"  Matched GO Term: {match['matched_go_name']} ({match['matched_go_id']})")
        print(f"  Similarity: {match['similarity']:.2f}")
        print(f"  Relationship: {match['relationship']}")
        print(f"  Namespace: {match['namespace']}")
        print(f"  Justification: {match['justification']}\n")

Gene: glial fibrillary acidic protein
  Llama Term: intermediate filament
  Matched GO Term: intermediate filament (GO:0005882)
  Similarity: 1.00
  Relationship: ['part_of GO:0045111']
  Namespace: cellular_component
  Justification: GFAP is expressed exclusively in astrocytes and is a type of intermediate filament protein.

  Llama Term: astrocyte differentiation
  Matched GO Term: astrocyte differentiation (GO:0048708)
  Similarity: 1.00
  Relationship: ['part_of GO:0007417']
  Namespace: biological_process
  Justification: GFAP is considered to be important for the induction and maintenance of astrocyte differentiation.

  Llama Term: regulation of cell proliferation
  Matched GO Term: regulation of cell population proliferation (GO:0042127)
  Similarity: 0.93
  Relationship: ['regulates GO:0008283']
  Namespace: biological_process
  Justification: Introduction of GFAP to C6 cells and human astrocytoma induced morphological changes, and suppressed their proliferation.

  Llama Term